In [4]:
import pandas as pd
from pandas import DataFrame as df
import os
import sqlite3 as db
import json

conn = db.connect('./test.db')
fda_df =pd.read_sql_query('select * from FDA', conn)
fda_df = fda_df[['지정번호','업소명','별점','소재지','지정기관구분','GPS']]

In [5]:
import requests as req
import json

def get_store_info(latitude, longitude, query_name):
# latitude = 35.1361642
# longitude = 126.9232147
# query = '수유리우동집'
    header = {'Authorization': 'KakaoAK 647742ba852c5d05653d9d6ca4666b70'}

    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    res = req.get(url, headers=header, params= {'page':1, 
                                'size':15,
                                'sort':'accuracy',
                                'category_group_code':'FD6',
                                'radius':10000,
                                'y':latitude,
                                'x':longitude,
                                'query':query_name,
                                })
    raw_json = json.loads(res.text)['documents']
    if len(raw_json) < 1:
        return {}
    store_Dic = raw_json[0]
    return store_Dic
    # print(store_Dic)
    # print(store_Dic['place_name'], store_Dic['id'])

In [6]:
gpsx = []
gpsy = []
for idx, item in fda_df['GPS'].iteritems():
    x,y = item.split(',')
    gpsx.append(x)
    gpsy.append(y)
fda_df['GPSX'] = pd.Series(gpsx)
fda_df['GPSY'] = pd.Series(gpsy)
del fda_df['GPS']
fda_df



,지정번호,업소명,별점,소재지,지정기관구분,GPSX,GPSY
0,HG180000183,나주곰탕 봉선관,★,"광주광역시 남구 봉선로 127(상가동 2층 211호 봉선동, 솔뫼)",식약처,35.1238744,126.9094705
1,HG180000562,The 온반,★★★,"광주광역시 동구 독립로 268(대인동, 10층)",식약처,35.1526693,126.9158064
2,HG180001161,청와삼대 풍암점,★★,광주광역시 서구 풍암2로 2(롯데슈퍼센터 2층 풍암동),식약처,35.1271516,126.8791112
3,HG180001175,황솔촌상무점,★★★,"광주광역시 서구 운천로 143(1층 쌍촌동, 외2필지)",식약처,35.1666591,126.8716265
4,HG180001190,본죽앤비빔밥,★,"광주광역시 서구 운천로 225(치평동,상무힐타워100호지하일층)",식약처,35.15248589999999,126.8544445
...,...,...,...,...,...,...,...
756,HG220006235,이디야커피 광주돌고개역점,★★★,"광주광역시 남구 구성로8번길 6(근린생활시설B동 1층 11호 월산동, 반도유보라 더...",식약처,35.1517133,126.8982625
757,HG220006239,파운데이 농성점,★★★,광주광역시 서구 대남대로 448(1층 농성동),식약처,35.1503173,126.8872441
758,HG220006241,BBQ광주주월점,★★★,"광주광역시 남구 대남대로223번길 10(110동 1층 110호 주월동, 양우 내안애...",식약처,35.1319935,126.896697
759,HG220006248,뚜레쥬르 효천블랙데이,★★★,광주광역시 남구 효우로 200(1층 101호 행암동),식약처,35.1041571,126.9030091


In [7]:
for lat,long,name in zip(fda_df['GPSX'], fda_df['GPSY'], fda_df['업소명']):
    tmp_dic = get_store_info(lat,long,name)

In [8]:
# Test code
tmp_df = fda_df[:10]
new_fda_list = []
for lat, long, name, hid, star, evl_fda in zip(tmp_df['GPSX'], tmp_df['GPSY'], tmp_df['업소명'], tmp_df['지정번호'], tmp_df['별점'], tmp_df['지정기관구분']):
    tmp_dic = get_store_info(lat,long,name)
    if len(tmp_dic) == 0:
        continue
    tmp_dic['evl_num'] = hid
    tmp_dic['evl_grade'] = star
    tmp_dic['evl_fda'] = evl_fda
    new_fda_list.append(tmp_dic)




In [9]:
new_fda_df = pd.read_json(json.dumps(new_fda_list, ensure_ascii=False))
new_fda_df

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,evl_num,evl_grade,evl_fda
0,광주 남구 봉선동 494,FD6,음식점,음식점 > 한식 > 곰탕,7,12319138,062-675-8250,나주곰탕 봉선관,http://place.map.kakao.com/12319138,광주 남구 봉선로 127,126.909477,35.123940,HG180000183,★,식약처
1,광주 서구 풍암동 1109-1,FD6,음식점,"음식점 > 한식 > 육류,고기 > 족발,보쌈",4,785629665,062-674-5200,청와삼대 광주풍암점,http://place.map.kakao.com/785629665,광주 서구 풍암2로 2,126.879083,35.127185,HG180001161,★★,식약처
2,광주 서구 쌍촌동 885-33,FD6,음식점,음식점 > 한식,2745,23736803,062-376-9233,황솔촌 상무점,http://place.map.kakao.com/23736803,광주 서구 운천로 143,126.856639,35.145190,HG180001175,★★★,식약처
3,광주 서구 치평동 1184-2,FD6,음식점,음식점 > 퓨전요리 > 퓨전한식 > 본죽&비빔밥cafe,648,7909440,062-376-6060,본죽&비빔밥cafe 광주상무점,http://place.map.kakao.com/7909440,광주 서구 치평로 80,126.847847,35.154666,HG180001190,★,식약처
4,광주 서구 치평동 1237-4,FD6,음식점,음식점 > 패밀리레스토랑 > 아웃백스테이크하우스,2,8474977,062-374-3761,아웃백스테이크하우스 광주상무점,http://place.map.kakao.com/8474977,광주 서구 상무연하로 41,126.849933,35.152004,HG180001244,★★★,식약처
5,광주 서구 화정동 1489,FD6,음식점,음식점 > 양식,3,332094368,,인더포레스트,http://place.map.kakao.com/332094368,광주 서구 군분로139번길 18,126.886586,35.144000,HG190000001,★★★,식약처
6,광주 남구 월산동 303-8,FD6,음식점,음식점 > 양식 > 이탈리안,43,1067575921,062-352-0674,루치아,http://place.map.kakao.com/1067575921,광주 남구 월산로116번길 22-1,126.899608,35.148515,HG190000501,★,식약처
7,광주 동구 대인동 7-1,FD6,음식점,"음식점 > 간식 > 제과,베이커리",17,22856480,062-221-0696,베비에르 롯데백화점 광주점,http://place.map.kakao.com/22856480,광주 동구 독립로 268,126.912094,35.154588,HG190000553,★★★,식약처


In [28]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
import time


addr='22856480'

def get_menu(driver):
        try:
             driver.find_element(By.CSS_SELECTOR, value='#mArticle > div.cont_menu > a > span.open_txt').click()
        except:
            pass
        menu_all = []
        try:
            menus = driver.find_element(By.CLASS_NAME, value='list_menu')
        
            menutitles = menus.find_elements(By.CLASS_NAME, value='info_menu')
            #menuprices = menus.find_elements(By.CLASS_NAME, value='price_menu')
            for menu in menutitles:
                menu_all.append(menu.text.replace('\n',':'))
        except:
            pass
        return menu_all

def get_pic(driver):
        image_urls = []
        pic_list = driver.find_elements(By.CLASS_NAME, value='link_photo')
        for ele in pic_list:
            raw_url = ele.get_attribute('style')
            raw_url = raw_url[raw_url.find('http'):raw_url.rfind('");')]
            raw_url = raw_url.replace('%2F','/')
            raw_url = raw_url.replace('%3A', ':')
            raw_url = raw_url.replace('%3F','?')
            image_urls.append(raw_url)
            # print(raw_url)
        return image_urls

def get_store_info_detail(store_id):
    driver = wb.Chrome('./chromedriver.exe')
    map_url = 'https://place.map.kakao.com/'+f'{store_id}'
    driver.get(map_url)
    time.sleep(2)

    try:
        storeNm = driver.find_element(By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2').text
    except:
        storeNm = "N/A"
    try:    
        storeAddr = driver.find_element(By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(2) > div > span.txt_address').text
    except:
        storeAddr = "N/A"
    try:
        storeTime = driver.find_element(By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div > div > ul > li:nth-child(1) > span > span').text
    except:
        storeTime = "N/A"
    try:
        storePhone = driver.find_element(By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div.details_placeinfo > div.placeinfo_default.placeinfo_contact > div > div > span > span.txt_contact').text
    except:
        storePhone = "N/A"
    storeMenus = get_menu(driver)
    storePics = get_pic(driver)

    store_detail_info = {
        "store_id" : store_id,
        "store_name" : storeNm,
        "store_tel" : storePhone,
        "store_addr" : storeAddr,
        "store_hours" : storeTime,
        "store_img" : ','.join(storePics),
        "store_menu" : ','.join(storeMenus),
    }
    return store_detail_info

# store_detail_info = get_store_info_detail(addr)
# store_detail_info

In [11]:
# new_fda_df
for store_id in new_fda_df['id']:
    print(store_id)

12319138
785629665
23736803
7909440
8474977
332094368
1067575921
22856480


{store_id,
store_name,
store_tel,
store_address,
store_hours,
store_images,
store_thumb,
store_lat,
store_long
}

최종 DB 스키마.

In [13]:
import sqlite3 as db
import json

conn = db.connect('./wesharingDataBase.db')
new_fda_df.to_sql("FDA", conn, if_exists="replace")



8

In [14]:
new_fda_df

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,evl_num,evl_grade,evl_fda
0,광주 남구 봉선동 494,FD6,음식점,음식점 > 한식 > 곰탕,7,12319138,062-675-8250,나주곰탕 봉선관,http://place.map.kakao.com/12319138,광주 남구 봉선로 127,126.909477,35.123940,HG180000183,★,식약처
1,광주 서구 풍암동 1109-1,FD6,음식점,"음식점 > 한식 > 육류,고기 > 족발,보쌈",4,785629665,062-674-5200,청와삼대 광주풍암점,http://place.map.kakao.com/785629665,광주 서구 풍암2로 2,126.879083,35.127185,HG180001161,★★,식약처
2,광주 서구 쌍촌동 885-33,FD6,음식점,음식점 > 한식,2745,23736803,062-376-9233,황솔촌 상무점,http://place.map.kakao.com/23736803,광주 서구 운천로 143,126.856639,35.145190,HG180001175,★★★,식약처
3,광주 서구 치평동 1184-2,FD6,음식점,음식점 > 퓨전요리 > 퓨전한식 > 본죽&비빔밥cafe,648,7909440,062-376-6060,본죽&비빔밥cafe 광주상무점,http://place.map.kakao.com/7909440,광주 서구 치평로 80,126.847847,35.154666,HG180001190,★,식약처
4,광주 서구 치평동 1237-4,FD6,음식점,음식점 > 패밀리레스토랑 > 아웃백스테이크하우스,2,8474977,062-374-3761,아웃백스테이크하우스 광주상무점,http://place.map.kakao.com/8474977,광주 서구 상무연하로 41,126.849933,35.152004,HG180001244,★★★,식약처
5,광주 서구 화정동 1489,FD6,음식점,음식점 > 양식,3,332094368,,인더포레스트,http://place.map.kakao.com/332094368,광주 서구 군분로139번길 18,126.886586,35.144000,HG190000001,★★★,식약처
6,광주 남구 월산동 303-8,FD6,음식점,음식점 > 양식 > 이탈리안,43,1067575921,062-352-0674,루치아,http://place.map.kakao.com/1067575921,광주 남구 월산로116번길 22-1,126.899608,35.148515,HG190000501,★,식약처
7,광주 동구 대인동 7-1,FD6,음식점,"음식점 > 간식 > 제과,베이커리",17,22856480,062-221-0696,베비에르 롯데백화점 광주점,http://place.map.kakao.com/22856480,광주 동구 독립로 268,126.912094,35.154588,HG190000553,★★★,식약처


In [31]:
store_info_detail_dicts = []
for store_id, gpsx, gpsy, evl_num, evl_grade, evl_fda in zip(new_fda_df['id'], new_fda_df['GPSX'], new_fda_df['GPSY'], new_fda_df['지정번호'], new_fda_df['별점'], new_fda_df['지정기관구분']):
    store_info_detail_dict = get_store_info_detail(store_id)
    store_info_detail_dict['store_lat'] = gpsx
    store_info_detail_dict['store_long'] = gpsy
    store_info_detail_dict['evl_num'] = evl_num
    store_info_detail_dict['evl_grade'] = evl_grade
    store_info_detail_dict['evl_fda'] = evl_fda
    store_info_detail_dicts.append(store_info_detail_dict)
store_info_detail_df = pd.read_json(json.dumps(store_info_detail_dicts, ensure_ascii=False))
store_info_detail_df.to_sql('tbl_store', conn, if_exists='replace')

C:\Users\GwangjuAI\AppData\Local\Temp\ipykernel_10728\3961450996.py:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wb.Chrome('./chromedriver.exe')


8